<a href="https://colab.research.google.com/github/cksdlakstp12/izone_finder/blob/main/image_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

function ClickConnect(){
    console.log("1분마다 코랩 연결 끊김 방지"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect-icon")
.click();
}
setInterval(ClickConnect, 6000 * 60);

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.0 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [ ]:
import cv2
import numpy as np
import time
import zipfile
import os
import urllib.request

from selenium.webdriver.chrome.options import Options 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from google.colab import drive

drive.mount("/content/drive")

KeyboardInterrupt: ignored

In [ ]:
IZ_ONE = {
    "최예나":"Choi_Yena", 
    "조유리":"Jo_Yuri",
    "김채원":"Kim_Chaewon", ####
    "안유진":"Ahn_Yujin", 
    "김민주":"Kim_Minjoo", ####
    "장원영":"Jang_Wonyoung",
    "나코":"Yabuki_Naco", 
    "히토미":"Honda_Hitomi", 
    "사쿠라":"Miyawaki_Sakura", 
    "이채연":"Lee_Chaeyeon", ####
    "강혜원":"Kang_Hyewon", 
    "권은비":"Kwon_Eunbi" #####
}

group_id_map = {
    "Choi_Yena":3281057791957156,
    "Jo_Yuri":567654570838415,
    "Kim_Chaewon":294558931717843, 
    "Ahn_Yujin":2530370073921773,
    "Kim_Minjoo":709696599827649,
    "Jang_Wonyoung":2678795852389874,
    "Yabuki_Naco":4040503105991235,
    "Honda_Hitomi":641973806405925,
    "Miyawaki_Sakura":611070823178096,
    "Lee_Chaeyeon":285602426013344, 
    "Kang_Hyewon":214761252930112,
    "Kwon_Eunbi":3135908313169431,
}

In [ ]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error : Creating directory. " + directory)

In [ ]:
for member_name in IZ_ONE.values():
    createFolder("/content/" + member_name)

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {"profile.default_content_setting_values.notifications":1}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

id = ""
pw = ""

main_page = "https://www."

driver.get(main_page)

login_box = driver.find_element_by_id("email")
login_box.send_keys(id)
login_box = driver.find_element_by_id("pass")
login_box.send_keys(pw)
login_box.send_keys(Keys.RETURN)

for eng_name, group_id in group_id_map.items():
  print(f"Start {eng_name}'s image crawling...'")
  
  time.sleep(3)
  driver.get(f"{main_page}groups/{group_id}/media")
  
  print("Start scrolling...")
  SCROLL_PAUSE_TIME = 1
  last_height = driver.execute_script("return document.body.scrollHeight")

  scroll_start_time = time.time()
  scroll_stop_time = 1200

  while 1:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    scroll_elapsed_time = time.time() - scroll_start_time
    if scroll_elapsed_time >= scroll_stop_time: break

  print("Start image downloading...", end="")
  images = driver.find_elements_by_css_selector(".k4urcfbm.bixrwtb6.datstx6m.q9uorilb")
  download_start_time = time.time()

  for i, img in enumerate(images):
    try:
      imgUrl = driver.find_element_by_xpath(f'/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div[4]/div/div/div/div/div/div/div/div[2]/div[{i+1}]/div/div/div/a/img').get_attribute("src")
      urllib.request.urlretrieve(imgUrl, f"/content/{eng_name}/{eng_name}_{i}.jpg")
    except:
      print("Unexpected error occured...")
      continue

  print(f" : done in {time.time() - download_start_time}")

  print("Start file compressing...")
  with zipfile.ZipFile(f'/content/{eng_name}_train.zip', 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
      
    ROOT_PATH = f"/content/{eng_name}/"
    img_name_list = os.listdir(ROOT_PATH)

    for img_name in img_name_list:
      new_zip.write(os.path.join(ROOT_PATH, img_name), arcname=img_name) 

  os.system(f"cp /content/{eng_name}_train.zip /content/drive/MyDrive/IZONE_finder/member_dataset/train")
  print(f"cp /content/{eng_name}_train.zip /content/drive/MyDrive/IZONE_finder/member_dataset/train")
  print("number of files : ", len(os.listdir(f"/content/{eng_name}")))
  print("="*50)

  continue

driver.close()

In [ ]:
import zipfile

for member_name in IZ_ONE.values():
  dir = f"/content/drive/MyDrive/IZONE_finder/member_dataset/{member_name}.zip"
  celeba = zipfile.ZipFile(dir)
  celeba.extractall(f"/content/{member_name")

celeba.close()

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

for kor_name, eng_name in IZ_ONE.items():
    print(eng_name)
    for k_i, keyword in enumerate(keywords):

        # if k_i < 15: continue

        try:
          driver.get("https://www.google.co.kr/imghp?hl=ko&tab=ri&ogbl")
          elem = driver.find_element_by_name("q")
          elem.send_keys(f"{group_name} {kor_name} {keyword}")
          elem.send_keys(Keys.RETURN)
          print(keyword)
        except:
          print(keyword + " is not able...")
          continue

        SCROLL_PAUSE_TIME = 1
        last_height = driver.execute_script("return document.body.scrollHeight")

        while 1:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height: 
                try:
                    driver.find_element_by_css_selector(".mye4qd").click()
                except:
                    break
            last_height = new_height

        images = driver.find_elements_by_css_selector(".rg_i.Q4LuWd")
        t0 = time.time()
        for i, img in enumerate(images):
            try:
                img.click()
                imgUrl = driver.find_element_by_xpath("/html/body/div[2]/c-wiz/div[3]/div[2]/div[3]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div[1]/a/img").get_attribute("src")
                urllib.request.urlretrieve(imgUrl, f"/content/{eng_name}/{eng_name}_{k_i}_{i}.jpg")
            except:
                pass
        print(f"One session done in {time.time() - t0}")
driver.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


Kwon_Eunbi
셀카
One session done in 567.0792827606201
라비앙로즈
One session done in 649.0313408374786
하늘위로
One session done in 160.78850483894348
스페이스쉽
One session done in 159.9916753768921
파노라마
One session done in 578.5192239284515
미장셴
One session done in 74.55953741073608
비올레타
One session done in 507.0338315963745
환상동화
One session done in 217.87841892242432
피에스타
One session done in 646.6567585468292
회전목마
One session done in 176.38819694519043
시퀀스
One session done in 532.4294710159302
에노지캠
One session done in 536.5094962120056
프로듀스
One session done in 480.83212399482727
잇힝트립
One session done in 189.87924242019653
브이앱
One session done in 579.3489825725555
아이즈원츄
One session done in 472.9644296169281
